In [67]:
import os
%pwd

'e:\\'

In [68]:
os.chdir("e:/End to End Wine Quality Prediction/")  # Replace with your actual previous path


In [69]:
%pwd

'e:\\End to End Wine Quality Prediction'

In [76]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    random_state: int
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    max_features: str
    max_depth: int
    bootstrap: bool
    target_column: str


In [77]:

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [79]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
     config = self.config.model_trainer
     model_params = self.params.best_hyperparameters
     model_name_config = self.params.model_name
     target_column_name = self.schema.TARGET_COLUMN.name

     create_directories([config.root_dir])

     model_trainer_config = ModelTrainerConfig(
        root_dir=config.root_dir,
        train_data_path=config.train_data_path,
        test_data_path=config.test_data_path,
        model_name=model_name_config.name,  # "RandomForestRegressor"
        random_state=model_name_config.random_state,  # 42
        n_estimators=model_params.n_estimators,  # 400
        min_samples_split=model_params.min_samples_split,  # 10
        min_samples_leaf=model_params.min_samples_leaf,  # 2
        max_features=model_params.max_features,  # "log2"
        max_depth=model_params.max_depth,  # 30
        bootstrap=model_params.bootstrap,  # True
        target_column=target_column_name
    )

     return model_trainer_config




In [80]:
import pandas as pd


In [81]:
import os
import joblib
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        #print("🟢 Starting model training...")  # Debugging step

        # Load training & testing data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        #print("✅ Data loaded successfully!")

        # Prepare input and target variables
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        #print("✅ Data preprocessing completed!")

        # Initialize and train the RandomForest model
        rf = RandomForestRegressor(
            n_estimators=self.config.n_estimators,
            min_samples_split=self.config.min_samples_split,
            min_samples_leaf=self.config.min_samples_leaf,
            max_features=self.config.max_features,
            max_depth=self.config.max_depth,
            bootstrap=self.config.bootstrap,
            random_state=self.config.random_state
        )

        #print("🚀 Training model...")
        rf.fit(train_x, train_y.values.ravel())  # Flatten y for sklearn
        #print("✅ Model training completed!")

        # Ensure the directory exists before saving
        os.makedirs(self.config.root_dir, exist_ok=True)

        # Define model path
        model_path = os.path.join(self.config.root_dir, self.config.model_name + ".joblib")
        #print(f"💾 Saving model at: {model_path}")

        # Save the trained model
        joblib.dump(rf, model_path)

        

  

In [82]:
# ✅ Pipeline Execution

try:
        config = ConfigurationManager()
        model_trainer_config = config.get_model_trainer_config()
        model_trainer = ModelTrainer(config=model_trainer_config)    
        model_trainer.train()
except Exception as e:
        raise e

[2025-03-27 23:39:01,809: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-27 23:39:01,821: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-27 23:39:01,834: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-27 23:39:01,839: INFO: common: created directory at: artifacts]
[2025-03-27 23:39:01,845: INFO: common: created directory at: artifacts/model_trainer]
